# Cálculo de Cohen d por grupo
Notebook para calcular el tamaño del efecto de Cohen d en variables binarias pre/post por grupo.

In [ ]:
from google.colab import files
uploaded = files.upload()
if uploaded:
    fname = next(iter(uploaded))
    if fname.lower().endswith('.dta'):
        df = pd.read_stata(fname)
    elif fname.lower().endswith(('.xlsx', '.xls')):
        df = pd.read_excel(fname)
    else:
        raise ValueError('Solo se admiten archivos .dta o .xlsx')
    display(df.head())


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def cohend_group(df: pd.DataFrame, varlist: list[str], group: str, method: str = 'dz') -> pd.DataFrame:
    """Calcula Cohen d para variables binarias pre y post por grupo.
    Devuelve un DataFrame con los resultados y exporta a Excel."""
    # 1. Validar columnas
    if group not in df.columns:
        raise ValueError(f'La columna de grupo {group} no existe en el DataFrame')
    if len(varlist) % 2 != 0:
        raise ValueError('varlist debe tener número par de variables')
    for col in varlist:
        if col not in df.columns:
            raise ValueError(f'La columna {col} no existe en el DataFrame')
    # 2. Dividir en pre y post
    mid = len(varlist) // 2
    prevars = varlist[:mid]
    postvars = varlist[mid:]
    df2 = df.copy()
    df2[prevars + postvars] = df2[prevars + postvars].apply(pd.to_numeric, errors='coerce')
    df2['tpre'] = (df2[prevars] == 2).mean(axis=1)
    df2['tpost'] = (df2[postvars] == 2).mean(axis=1)
    df2['dif'] = df2['tpost'] - df2['tpre']
    resultados = []
    for nivel in df2[group].astype(str).unique():
        sub = df2[df2[group].astype(str) == str(nivel)].dropna(subset=['tpre','tpost'])
        N = len(sub)
        if N == 0:
            continue
        m_pre = sub['tpre'].mean()
        m_post = sub['tpost'].mean()
        m_diff = sub['dif'].mean()
        sd_pre = sub['tpre'].std(ddof=1)
        sd_post = sub['tpost'].std(ddof=1)
        sd_diff = sub['dif'].std(ddof=1)
        r_pp = sub['tpre'].corr(sub['tpost'])
        if method == 'dz':
            denom = sd_diff
        elif method == 'dav':
            denom = (sd_pre + sd_post) / 2
        elif method == 'drm':
            denom = np.sqrt(sd_pre**2 + sd_post**2 - 2*r_pp*sd_pre*sd_post)
        else:
            raise ValueError('method debe ser "dz", "dav" o "drm"')
        d_cohen = m_diff / denom if denom and denom != 0 else np.nan
        if N >= 2:
            p_value = stats.ttest_rel(sub['tpost'], sub['tpre'], nan_policy='omit').pvalue
        else:
            p_value = np.nan
        if pd.isna(d_cohen):
            magnitude = 'NA'
            symbol = ''
        elif abs(d_cohen) >= 0.80:
            magnitude = 'GRANDE'
            symbol = '★★★'
        elif abs(d_cohen) >= 0.50:
            magnitude = 'MEDIANO'
            symbol = '★★'
        elif abs(d_cohen) >= 0.20:
            magnitude = 'PEQUEÑO'
            symbol = '★'
        else:
            magnitude = 'TRIVIAL'
            symbol = ''
        resultados.append({'grupo': nivel,'N': N,'m_pre': m_pre,'m_post': m_post,'m_diff': m_diff,'d_cohen': d_cohen,'p_value': p_value,'magnitude': magnitude,'symbol': symbol})
    result_df = pd.DataFrame(resultados)
    filename = 'cohend_resultados.xlsx'
    titulo = f"Cohen d – Variables: {', '.join(prevars)}, {', '.join(postvars)} (método={method})"
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        result_df.to_excel(writer, sheet_name='resultados', startrow=2, index=False)
        writer.sheets['resultados'].write('A1', titulo)
    print(f'Archivo guardado: {filename}')
    return result_df


In [ ]:
import ipywidgets as widgets
from IPython.display import display

def interfaz_cohend(df):
    columnas = [c for c in df.columns if c != 'macro']
    sel = widgets.SelectMultiple(options=columnas, description='Variables')
    metodo = widgets.Dropdown(options=['dz','dav','drm'], value='dz', description='Método')
    boton = widgets.Button(description='Calcular')
    out = widgets.Output()
    
    def al_hacer_click(b):
        with out:
            out.clear_output()
            varlist = list(sel.value)
            if len(varlist) % 2 != 0:
                print('Seleccione un número par de variables')
                return
            res = cohend_group(df, varlist, group='macro', method=metodo.value)
            display(res)
    boton.on_click(al_hacer_click)
    display(sel, metodo, boton, out)

# Ejecutar después de cargar el DataFrame 'df'
interfaz_cohend(df)
